In [ ]:
from glob import glob
import logging
import os
from pprint import pprint

import geojson
import fiona
from shapely.geometry import mapping, shape, box

logger = logging.getLogger(__name__)


In [ ]:
# WGS84
shp_path = os.path.join(os.environ["DATA_DIR"], 'shapefile_out')
layer_path = os.path.join(os.environ["DATA_DIR"], 'landgrid', 'DI_basemaps_WGS84.gdb')


## Read Shapefile

In [ ]:

with fiona.open(os.path.join(shp_path, 'Counties_US_WGS84_shgrid.shp')) as c_iter:
    print(f"Driver: {c_iter.driver}, CRS: {c_iter.crs}")
    print(f"Num Records: {len(c_iter)}")
    
    rec = next(iter(c_iter))
    pprint(f"SCHEMA: {c_iter.schema}")
    pprint(rec.keys())
    pprint(rec['type'])
    pprint(rec['id'])
    pprint(rec['properties'])
    pprint(rec['geometry'])

    # for rec in c_iter:
    #     pprint(rec['type'])
    #     pprint(rec['id'])
    #     pprint(rec['properties'])
    #     pprint(rec['geometry'])
    
    pprint(f"Access by index: {c_iter[21]}")
    

In [ ]:

batches = glob(os.path.join(shp_path, f'*_township*.shp'))

for b in batches:
    with fiona.open(b) as c_iter:
        print(f"Num Records {os.path.split(b)[1]}: {len(c_iter)}")


## Read ESRI File Geodatabase
- See https://gdal.org/drivers/vector/filegdb.html
- See https://fiona.readthedocs.io/en/stable/manual.html#reading-multilayer-data

In [ ]:

# List layers in GDB
for name in sorted(fiona.listlayers(layer_path)):
    print(name)

In [ ]:
layer_path = os.path.join(os.environ["DATA_DIR"], 'landgrid', 'DI_basemaps_WGS84.gdb')

with fiona.open(layer_path, layer='States_US') as c_iter:
    print(f"Driver: {c_iter.driver}, CRS: {c_iter.crs}")
    print(f"Num Records: {len(c_iter)}")
    pprint(f"SCHEMA: {c_iter.schema}")
    
    rec = next(iter(c_iter))
    pprint(rec.keys())
    pprint(rec['type'])
    pprint(rec['id'])
    pprint(rec['properties'])
    pprint(rec['geometry'])

    # for rec in c_iter:
    #     pprint(rec['type'])
    #     pprint(rec['id'])
    #     pprint(rec['properties'])
    #     pprint(rec['geometry'])


## Write Shapefile

In [ ]:
%%time

# WGS84 (epsg:4326)
gdb_src_path = os.path.join(os.environ["DATA_DIR"], 'landgrid', 'DI_basemaps_WGS84.gdb')
shp_out_path = os.path.join(os.environ["DATA_DIR"], 'shapefile_out', 'test_fiona.shp')

with fiona.open(gdb_src_path, layer='States_US') as src:
    print(f"Driver: {src.driver}, CRS: {src.crs}")
    
    # Copy the source schema and add a property.
    schema = src.schema.copy()
    schema['properties']['new_prop'] = 'int'  # new
    schema['geometry'] = 'Polygon'  # update
    print(f"Schema: {schema}")
    
    with fiona.open(shp_out_path, 'w', driver='ESRI Shapefile', 
                    schema=schema, crs=src.crs) as tgt:
        
        i = 0
        for f in src:
            #print(f"Coordinates: {f['geometry']['coordinates']}")
            
            # See https://shapely.readthedocs.io/en/latest/manual.html#python-geo-interface
            state = shape(f['geometry'])
            print(f"Type: {f['geometry']['type']}, State: {f['properties']['State_Name']}")
            #print(f"Bounds: {state.bounds}")
            
            if not state.is_valid:
                clean_state = state.buffer(0.0)
                assert clean_state.is_valid
                assert clean_state.geom_type == 'MultiPolygon'
                state = clean_state
            
            bbox = box(*state.bounds)
            
            # Writes as python-geo-interface (GeoJSON-like)
            f['geometry'] = mapping(bbox)
            pprint(f"Geometry: {f['geometry']}")
            
            # Create GeoJSON representation.
            pprint(f"GeoJSON: {geojson.dumps(f['geometry'], sort_keys=True)}")
            f['properties'].update(new_prop=i)
            i += 1
            
            tgt.write(f)
